In [ ]:
!pip install openmeteo-requests

In [ ]:
!pip install requests-cache retry-requests

In [ ]:
!pip install global-land-mask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.9 MB/s eta 0:00:00


In [ ]:
import asyncio
import aiohttp
import nest_asyncio
import pandas as pd
from typing import List, Tuple
from global_land_mask import globe
import numpy as np
from datetime import datetime
from retry_requests import retry
import openmeteo_requests
import requests
import requests_cache
import geopandas as gpd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class TreeRingData(Dataset):

  def __init__(self):

    self.endpoints = self.__get_tree_rings()
    self.data_dump = []
    nest_asyncio.apply()
    asyncio.get_event_loop().run_until_complete(self.__fill_data())

    self.dataframes = []

    for j in self.endpoints:
      cols = True
      df = None
      for i in j.splitlines():
        if i[0] != '#':
          temp = i.split()
          if cols:
            df = pd.DataFrame(columns=temp)
            cols = False
          else:
            df.loc[len(df.index)] = temp

      self.dataframes.append(df)

    print(len(self.dataframes[0]))

  async def __fill_data(self):
    self.data_dump = await self.__caller(self.endpoints)

  async def __fetch(self, session, url):
    async with session.get(url) as response:
      return await response.json()

  async def __caller(self, urls):
    async with aiohttp.ClientSession() as session:
      tasks = [self.__fetch(session, url) for url in urls]
      resps = await asyncio.gather(*tasks, return_exceptions=True)
      return resps

  def __get_tree_rings(self):

    api = "https://www.ncei.noaa.gov/access/paleo-search/study/search.json"
    response = requests.get(api, params={
        "dataTypeId": 18,
        "continent": "North America"
    })

    if response.status_code == 200:

      allowed_names = {
          "Pinus ponderosa Douglas ex C. Lawson": 0,
          "Pinus sylvestris L.": 0,
          "Pinus edulis Engelm.": 0
      }

      response = response.json()
      urls = set()
      for i in response["study"]:
        for j in i["site"]:
          for k in j["paleoData"]:

            includes_pinus = False
            for o in k["species"]:
              if o["scientificName"] in allowed_names:
                allowed_names[o["scientificName"]]+=1
                includes_pinus = True

            if includes_pinus:
              for l in k["dataFile"]:
                urls.add(l["fileUrl"])

      print(allowed_names)
      return urls
    else:
      raise Exception("Could not reach Paleo Data. Please ensure a stable connection.")

In [ ]:
class WeatherAPI:


  def __init__(self, date_start, date_end, generate_num=100000, expire_after=-1, retries=5, backoff_factor=.2):

    # Create Session with OpenMeteo, establish Endpoint
    self.cache = requests_cache.CachedSession('.cache', expire_after=expire_after)
    self.OpenMeteo = openmeteo_requests.Client(session=retry(self.cache, retries=retries, backoff_factor=backoff_factor))
    self.endpoint = "https://archive-api.open-meteo.com/v1/archive"

    # self.dates = self.__process_dates(date_start, date_end)

    self.start_date = f"{date_start}-01-01"
    self.end_date = f"{date_end}-12-31"

    self.endpoints = ["https://api.3geonames.org/?randomland=US&json=1"]*generate_num
    self.data_dump = []
    nest_asyncio.apply()
    asyncio.get_event_loop().run_until_complete(self.__fill_data())

    long_lat = []
    for i in self.data_dump:
      long_lat.append((i["nearest"]["latt"], i["nearest"]["longt"]))

    print(long_lat)

  async def __fill_data(self):
    self.data_dump = await self.__caller(self.endpoints)

  async def __fetch(self, session, url):
    async with session.get(url) as response:
      return await response.json()

  async def __caller(self, urls):
    async with aiohttp.ClientSession() as session:
      tasks = [self.__fetch(session, url) for url in urls]
      resps = await asyncio.gather(*tasks, return_exceptions=True)
      return resps

  # def __process_dates(self, year_start: int, year_end: int) -> List[Tuple[int, int]]:

  #   f = "%m/%d/%Y"
  #   dates_list = [
  #     (int(datetime.strptime(f"01/01/{i}", f).timestamp()), int(datetime.strptime(f"12/31/{i}", f).timestamp())) for i in range(year_start, year_end+1)
  #   ]

  #   return dates_list


  # def __send_request(self, packet: dict, verbose=False) -> pd.DataFrame:

  #   reponse = None
  #   df = pd.DataFrame()
  #   start_year, end_year = packet['year_start'], packet['year_end']
  #   dates = self.__process_dates(start_year, end_year)
  #   packet['start_date'] = f"{packet['year_start']}-01-01"
  #   packet['end_date'] = f"{packet['year_end']}-12-31"

  #   if verbose:
  #     print(packet)

  #   try:
  #     response = self.OpenMeteo.weather_api(self.endpoint, params=packet)[0]
  #   except Exception as e:
  #     if verbose:
  #       print(e)
  #     return df

  #   if verbose:
  #     print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
  #     print(f"Elevation {response.Elevation()} m asl")
  #     print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
  #     print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

  #   daily = response.Daily()
  #   precip = daily.Variables(0).ValuesAsNumpy()

  #   last_ind = 0
  #   df_data = {
  #     "year": [],
  #     "precipitation_sum": [],
  #     "precipitation_daily_avg": [],
  #     "precipitation_min": [],
  #     "precipitation_max": []
  #   }

  #   for i in range(0, end_year-start_year+1):

  #     days = len(pd.date_range(
  #       start = pd.to_datetime(dates[i][0], unit = "s"),
  #       end = pd.to_datetime(dates[i][1], unit = "s"),
  #       freq = pd.Timedelta(seconds = 86400),
  #       inclusive = "left"
  #     ))

  #     years_data = precip[last_ind:days+last_ind]
  #     df_data["year"].append(i+start_year)
  #     df_data["precipitation_sum"].append(years_data.sum())
  #     df_data["precipitation_daily_avg"].append(np.mean(years_data))
  #     df_data["precipitation_min"].append(np.min(years_data))
  #     df_data["precipitation_max"].append(np.max(years_data))

  #     last_ind=days+last_ind

  #   dataframe = pd.DataFrame(data=df_data)

  #   return dataframe


  # def get_yearly_precipitation(self, year_start: int, year_end: int, long_lat: Tuple, verbose=False) -> pd.DataFrame:
  #   return self.__send_request({
  #     "latitude": long_lat[1],
  #     "longitude": long_lat[0],
  #     "year_start": year_start,
  #     "year_end": year_end,
  #     "daily": "precipitation_sum"
  #   }, verbose=verbose)

In [ ]:
a = WeatherAPI(2022, 2023)

TypeError: ignored

In [ ]:
a.data_dump

In [ ]:
# grab shape within which to sample
url = "https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_nation_20m.zip"
us = gpd.read_file(url).explode()
## filter out parts of the US that are far away from mainland, I have no idea what they are (Guam islands?)
us = us.loc[us.geometry.apply(lambda x: x.exterior.bounds[2])<-60]

# grab bounding box within which to generate random numbers
x_min,y_min,x_max,y_max = us.geometry.unary_union.bounds

# the sampling
np.random.seed(2) # set seed (needed for reproducible results
N = 100000


<ipython-input-18-3cbfb002b1c0>:3: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  us = gpd.read_file(url).explode()


KeyboardInterrupt: ignored

In [73]:
top, bottom = 49, 25
left, right = -124, -67

MILE_DIFF = .4

points = []
for i in np.arange(25, 49, MILE_DIFF):
  numbers = np.arange(-124, -67, MILE_DIFF)
  for j in numbers:
    if globe.is_land(i, j):
      points.append((i, j))

len(points)

6204

In [74]:
df = pd.DataFrame(points, columns=["Lat", "Long"])

In [75]:
df.to_csv('/content/drive/MyDrive/points.csv')